<a href="https://colab.research.google.com/github/Progressive-Programmer/image_to_ttf/blob/main/image_to_ttf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import os
import glob

# Get a list of all files and directories in the current directory
files_and_dirs = glob.glob('*')
file_to_keep = 'input_image.jpeg'

for item in files_and_dirs:
    # Skip the specific file we want to keep
    if item == file_to_keep:
        print(f"Skipping: {item} (kept as requested)")
        continue

    try:
        if os.path.isfile(item):
            os.remove(item)
            print(f"Deleted file: {item}")
        elif os.path.isdir(item):
            shutil.rmtree(item)
            print(f"Deleted directory: {item}")
        else:
            print(f"Skipping unknown item type: {item}")
    except Exception as e:
        print(f"Error deleting {item}: {e}")

Skipping: input_image.jpeg (kept as requested)
Deleted file: script_Regular.py
Deleted file: LogoFont_Family.ttc
Deleted file: optimized_build_fixed.py
Deleted file: script_Italic.py
Deleted file: build_Black.py
Deleted directory: png_deep_clean
Deleted file: LogoFont_Thin.otf
Deleted file: LogoFont_Smooth_Regular.ttf
Deleted file: script_Thin.py
Deleted file: build_Italic.py
Deleted file: LogoFont_Thin.ttf
Deleted directory: svg_Regular
Deleted directory: svg_Black
Deleted file: script_Black.py
Deleted file: LogoFont_Smooth_Bold.ttf
Deleted file: build_BoldItalic.py
Deleted directory: svg_final_v4
Deleted file: LogoFont_Black.ttf
Deleted file: build_Thin.py
Deleted directory: png_smooth_prep
Deleted file: build_Bold.py
Deleted file: LogoFont_Bold.otf
Deleted file: LogoFont_Black.otf
Deleted directory: svg_Bold
Deleted directory: svg_glyphs_input
Deleted directory: svg_family_cache
Deleted file: LogoFont_Final_Family.ttc
Deleted file: script_Bold.py
Deleted file: LogoFont_Regular.ttf
D

In [ ]:
import cv2
import numpy as np
import os

def process_font_grid_v2(image_path, rows=7, cols=6, target_size=512):
    # Load the image from the specified path.
    img = cv2.imread(image_path)
    # Convert the image to grayscale, which simplifies processing for finding shapes.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply an adaptive threshold to convert the grayscale image into a black and white image.
    # This helps in isolating the characters from the background.
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 21, 10)

    # Get the height and width of the processed image.
    h_img, w_img = thresh.shape
    # Calculate the approximate height and width of each cell in the grid, based on the number of rows and columns.
    cell_h, cell_w = h_img // rows, w_img // cols

    # Define an output folder where the cleaned PNG images of individual glyphs (characters) will be saved.
    output_folder = "svg_glyphs_input" # New folder for clean PNGs
    # Create the output folder if it doesn't already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Define a margin to effectively remove any grid lines that might be present in the original image.
    # This margin cuts off a small percentage (10%) from the edges of each cell.
    margin_y = int(cell_h * 0.10)
    margin_x = int(cell_w * 0.10)

    char_idx = 0 # Initialize a counter for naming the glyph files.
    # Loop through each row of the grid.
    for r in range(rows):
        # Loop through each column of the grid.
        for c in range(cols):
            # Calculate the coordinates (top-left and bottom-right) for the current cell, adjusted by the margin.
            y1, y2 = r * cell_h + margin_y, (r + 1) * cell_h - margin_y
            x1, x2 = c * cell_w + margin_x, (c + 1) * cell_w - margin_x
            # Extract the cell (potential character) from the thresholded image.
            cell = thresh[y1:y2, x1:x2]

            # Find contours (outlines) within the extracted cell.
            # This helps identify individual character shapes.
            cnts, _ = cv2.findContours(cell, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if cnts:
                x_c, y_c = [], []
                # Iterate through the found contours.
                for cnt in cnts:
                    # Only consider contours that are large enough to be actual characters, ignoring small noise.
                    if cv2.contourArea(cnt) > 100: # Higher threshold to ignore tiny noise
                        # Get the bounding box (rectangle) around the contour.
                        bx, by, bw, bh = cv2.boundingRect(cnt)
                        # Store the coordinates of the bounding box.
                        x_c.extend([bx, bx + bw]); y_c.extend([by, by + bh])

                if x_c:
                    # Crop the character tightly based on the combined bounding boxes of its parts.
                    char_crop = cell[min(y_c):max(y_c), min(x_c):max(x_c)]

                    # Normalize the character's size and add padding to prevent cut-offs during vectorization.
                    # This ensures characters are consistently sized and centered.
                    h_c, w_c = char_crop.shape # Get the dimensions of the cropped character.
                    inner_size = int(target_size * 0.6) # Define a smaller 'inner' size for the character itself within the canvas.
                    # Calculate the scaling factor to fit the character into the 'inner_size'.
                    scale = inner_size / max(h_c, w_c)
                    # Calculate the new width and height after scaling.
                    nw, nh = int(w_c * scale), int(h_c * scale)
                    # Resize the character using the calculated dimensions.
                    resized = cv2.resize(char_crop, (nw, nh), interpolation=cv2.INTER_AREA)

                    # Create a blank square canvas of the target_size.
                    canvas = np.zeros((target_size, target_size), dtype=np.uint8)
                    # Calculate offsets to center the resized character on the canvas.
                    x_off, y_off = (target_size - nw) // 2, (target_size - nh) // 2 # Center both ways
                    # Place the resized character onto the center of the canvas.
                    canvas[y_off:y_off+nh, x_off:x_off+nw] = resized

                    # Save the cleaned and normalized character image as a PNG file in the output folder.
                    cv2.imwrite(f"{output_folder}/glyph_{char_idx:03d}.png", canvas)
                    char_idx += 1 # Increment the character counter.
    print(f"Cleaned {char_idx} glyphs with extra padding.")

# Call the function to process the input image.
# Make sure 'input_image.jpeg' exists in the same directory or provide its full path.
process_font_grid_v2('input_image.jpeg')

Cleaned 42 glyphs with extra padding.


In [ ]:

!apt-get update
!apt-get install -y potrace fontforge

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,286 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
potrace is already the newest version (1.16-2).
fo

In [ ]:
# Install system-level font tools
!apt-get update
!apt-get install -y fontforge potrace

# Install python-level font libraries
!pip install fonttools[ufo,lxml] -q


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (1,787 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
potrace is already the newest version (1.16-2).
fo

In [ ]:
import os
import subprocess
import cv2
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from fontTools.ttLib import TTCollection, TTFont
from google.colab import files

# --- 1. CONFIG ---
input_png_folder = "svg_glyphs_input"
base_svg_dir = "svg_family_cache"
os.makedirs(base_svg_dir, exist_ok=True)

# Corrected Sequence for White-on-Black inputs:
# (Suffix, Action, Strength, Slant)
# Slant 0.2 is the standard for Italics.
variants = [
    ("Thin",        "erode",  3, 0),
    ("Regular",     "none",   0, 0),
    ("Bold",        "dilate", 4, 0),
    ("Black",       "dilate", 9, 0),
    ("Italic",      "none",   0, 0.2),
    ("BoldItalic",  "dilate", 4, 0.2)
]

# --- 2. THE FAST PIXEL & VECTOR ENGINE ---
def process_weight(variant):
    suffix, action, strength, slant = variant
    weight_svg_dir = os.path.join(base_svg_dir, suffix)
    os.makedirs(weight_svg_dir, exist_ok=True)

    png_files = sorted([f for f in os.listdir(input_png_folder) if f.endswith('.png')])

    for filename in png_files:
        img = cv2.imread(os.path.join(input_png_folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is None: continue

        # Binary Threshold (Input is already white-on-black)
        _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

        # APPLY WEIGHT (Operating on White pixels)
        if action != "none":
            kernel = np.ones((3,3), np.uint8)
            if action == "dilate":
                binary = cv2.dilate(binary, kernel, iterations=strength)
            else:
                binary = cv2.erode(binary, kernel, iterations=strength)

        # SQUARE FIX: Add 60px of Black padding around the White logo
        padded = cv2.copyMakeBorder(binary, 60, 60, 60, 60, cv2.BORDER_CONSTANT, value=0)

        # POTRACE FIX: Flip to Black Logo on White Background
        final_for_potrace = cv2.bitwise_not(padded)

        temp_bmp = f"temp_{suffix}_{filename}.bmp"
        cv2.imwrite(temp_bmp, final_for_potrace)

        # VECTORIZE
        name = os.path.splitext(filename)[0]
        svg_path = os.path.join(weight_svg_dir, f"{name}.svg")
        subprocess.run(["potrace", "--svg", "--flat", "-t", "50", temp_bmp, "-o", svg_path])
        os.remove(temp_bmp)

    # --- 3. THE LEAN FONT ASSEMBLY ---
    otf_path = f"LogoFont_{suffix}.ttf"
    ff_script = f"""
import fontforge
import os
font = fontforge.font()
font.familyname, font.fontname = "LogoFont", "LogoFont-{suffix}"
svg_dir = "{weight_svg_dir}"
svg_files = sorted([f for f in os.listdir(svg_dir) if f.endswith('.svg')])
chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789!@#$%^&*()-_=+"

for i, filename in enumerate(svg_files):
    if i >= len(chars): break
    glyph = font.createChar(ord(chars[i]))
    glyph.importOutlines(os.path.join(svg_dir, filename))

    # Apply Italic Slant if needed
    if {slant} != 0:
        glyph.transform([1, 0, {slant}, 1, 0, 0])

    glyph.simplify()
    glyph.transform([0.75, 0, 0, 0.75, 0, 0])
    bbox = glyph.boundingBox()
    # Centering logic
    glyph.transform([1, 0, 0, 1, -bbox[0] + (1000-(bbox[2]-bbox[0]))/2, -bbox[1]])
    glyph.width = 1000
font.generate("{otf_path}")
"""
    with open(f"build_{suffix}.py", "w") as f: f.write(ff_script)
    subprocess.run(["fontforge", "-script", f"build_{suffix}.py"], capture_output=True)
    return otf_path

# --- 4. EXECUTION ---
print("🚀 Executing Parallel Build for 6 variants...")
with ProcessPoolExecutor() as executor:
    generated_files = list(executor.map(process_weight, variants))

# --- 5. STITCH & DOWNLOAD ---
print("🧵 Stitching into Family Collection...")
collection = TTCollection()
for f in generated_files:
    if os.path.exists(f): collection.fonts.append(TTFont(f))
collection.save("LogoFont_Complete_Family.ttc")
files.download("LogoFont_Complete_Family.ttc")
print("✅ Done! Your full font family is ready.")

🚀 Executing Parallel Build for 6 variants...
🧵 Stitching into Family Collection...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Done! Your full font family is ready.
